In [ ]:
from calibration_functions import *
from typing import List, Tuple

import cv2
import json
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import os

In [ ]:
with open('../config.json') as f:
    config = json.load(f)
BASE_DIR = config["BASE_DIR"]
VIS_DIR = config["BASE_DIR_VIS"]

In [ ]:
image_path1 = os.path.join(BASE_DIR, 'data_3/ainstec/000/t.bmp')
image_path2 = os.path.join(BASE_DIR, 'data_3/multicast_rect_reso/camera1/image_000.png')

rvec, tvec = find_and_compute_transform(image_path1, image_path2, (6, 10), imshow=False)

print(rvec)
print(tvec)

print(cv2.Rodrigues(rvec))

In [ ]:
image_path1 = os.path.join(BASE_DIR, 'data_3/ainstec/000/t.bmp')
image_path2 = os.path.join(BASE_DIR, 'data_3/multicast_rect_reso/camera1/image_000.png')

image1 = cv2.imread(image_path1)
image2 = cv2.imread(image_path2)

# plt.imshow(image1)
# plt.show()
# plt.imshow(image2)
# plt.show()

H, mask = find_homography_chessboard(image1, image2, (6, 10))

print(H)


In [ ]:
def main_folder():
    scan_folder        = os.path.join(BASE_DIR, 'data_2/ainstec')
    image_folder_left  = os.path.join(BASE_DIR, 'data_2/multicast_rect_reso/camera1')
    image_folder_right = os.path.join(BASE_DIR, 'data_2/multicast_rect_reso/camera2')
    output_path        = os.path.join(BASE_DIR, 'data_2/ground_truth_pfm_rect')

    os.makedirs(output_path, exist_ok=True)

    list_of_scans_folders: List[str] = sorted(os.listdir(scan_folder))
    list_of_left_images:   List[str] = sorted([os.path.join(image_folder_left , x) for x in os.listdir(image_folder_left ) if x != '.DS_Store'])
    list_of_right_images:  List[str] = sorted([os.path.join(image_folder_right, x) for x in os.listdir(image_folder_right) if x != '.DS_Store'])

    list_of_scans_paths: List[str] = []
    for subfolder in list_of_scans_folders:
        if str(subfolder) == ".DS_Store":
            continue
        folder_path = os.path.join(scan_folder, subfolder)
        pcd_files = [os.path.join(folder_path, x) for x in os.listdir(folder_path) if x.endswith('.pcd')]
        list_of_scans_paths.extend(pcd_files)

    # Calibration data
    scan_calib_image_path = [os.path.join(BASE_DIR, 'data_3/ainstec/000/t.bmp')]
    scan_calib_pcd_path   = [os.path.join(BASE_DIR, 'data_3/ainstec/000/t.pcd')]
    left_calib_image_path = [os.path.join(BASE_DIR, 'data_3/multicast_rect_reso/camera1/image_000.png')]

    image1 = cv2.imread(scan_calib_image_path[0])
    image2 = cv2.imread(left_calib_image_path[0])


    if image1 is None or image2 is None:
        print("Error: One of the images failed to load.")
        return

    dimensions = (image2.shape[1], image2.shape[0])

    rvec, tvec = find_and_compute_transform(scan_calib_image_path[0], left_calib_image_path[0], (6, 10), imshow=False)
    H: np.ndarray
    H, _ = find_homography_chessboard(image1, image2, (6, 10))
    
    mask: np.ndarray
    mask = generate_original_image_mask(image1, H, dimensions)

    if H is None:
        print("Error: Failed to compute homography.")
        return

    combined_pcd_paths: List[str]    = scan_calib_pcd_path   + list_of_scans_paths
    combined_image1_paths: List[str] = scan_calib_image_path + list_of_left_images
    combined_image2_paths: List[str] = left_calib_image_path + list_of_right_images

    for count, (scan_path, image_1_path, image_2_path) in enumerate(zip(combined_pcd_paths, combined_image1_paths, combined_image2_paths)):
        count -= 1
        print(scan_path)
        print(image_1_path)
        print(image_2_path)
        # if count < 0:
        #     continue
        # if count == np.inf:
        #     break

        image1 = cv2.imread(image_1_path)
        image2 = cv2.imread(image_2_path)

        pcd = o3d.io.read_point_cloud(scan_path)
        # corrected_pcd = apply_transformation_to_point_cloud(pcd, rvec, tvec)
        corrected_pcd     = apply_advanced_perspective_correction(pcd, H)

        dimensions = (1624, 1240)

        z_matrix = np.zeros((dimensions[1], dimensions[0]))
        points   = np.asarray(corrected_pcd.points)

        lengths = np.linalg.norm(points, axis=1)

        for idx, point in enumerate(points):
            x_idx = idx % dimensions[0]
            y_idx = idx // dimensions[0]
            z_matrix[y_idx, x_idx] = lengths[idx]

        warp_image_1 = warp_image(image1, H, dimensions)
        warped_depth = warp_image(z_matrix, H, dimensions)

        save_calibration_image = False
        if save_calibration_image == True:
            fig, ax = plt.subplots(2,3 , figsize=(18, 10), dpi = 300)

            ax[0, 0].imshow(image1)
            ax[0, 0].set_title('3D scan image')

            ax[1, 0].imshow(image2)
            ax[1, 0].set_title('left camera reference Image')

            ax[0, 1].imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
            ax[0, 1].imshow(mask, cmap='gray', alpha=0.5)
            ax[0, 1].set_title('validity mask overlay')

            ax[1, 1].imshow(warp_image_1)
            ax[1, 1].set_title('3D scan image warped')

            ax[0, 2].imshow(z_matrix)
            ax[0, 2].set_title('point cloud depth')

            ax[1, 2].imshow(warped_depth)
            ax[1, 2].set_title('corrected warped depth')

            plt.tight_layout()
            plt.show()
            SAVE_PARH = os.path.join(VIS_DIR, f'dataset_1/vis_{count:03}.png')
            plt.savefig(SAVE_PARH)

            if count == 1:
                break
        
        save_dataset_visualization = True
        if save_dataset_visualization == True:
            if count == 1:
                continue

            fig, ax = plt.subplots(1,3 , figsize=(18, 5), dpi = 300)

            ax[0].imshow(warped_depth)
            ax[0].set_title('point cloud depth')

            ax[1].imshow(image1)
            ax[1].set_title('left camera reference Image')

            ax[2].imshow(image2)
            ax[2].set_title('right camera')
            
            plt.tight_layout()
            SAVE_PARH = os.path.join(VIS_DIR, f'dataset_2/vis_{count:03}.png')
            plt.savefig(SAVE_PARH)
            plt.show()

        save_ground_truth_rect = False
        if save_ground_truth_rect == True:
            if count >= 0:
                pass
            else:
                continue
            # plt.imsave(os.path.join(output_path, f'ground_truth_{count:03}.png'), warped_depth)
            warped_depth_float32 = warped_depth.astype(np.float32)
            write_pfm(os.path.join(output_path, f'ground_truth_{count:03}.pfm'), warped_depth_float32)
            print(f"Saved ground truth {count:03}.pfm")

        print(count)

In [ ]:
if __name__ == "__main__":
    main_folder()